# join_2dataframes

In [1]:
%load_ext autoreload
%autoreload 2

# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload

In [2]:
import os
import sys
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:8,.2f}'.format

In [3]:
# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)

In [4]:
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.sql.window import Window


# spark = SparkSession.builder.appName('myappname').getOrCreate()
# print(spark)

In [5]:
# print(dir(pyspark))
# print(dir(pyspark.sql))
# print(dir(pyspark.rdd))

In [6]:
from shared.app_context import *

ctx = ApplicationContext("Dev-Job")
print(ctx.spark)

In [7]:
# print(sys.path)
from builder.DataFrameBuild import *

## DataFrameBuild Class:
### x = DataFrameBuild(ctx.spark)

In [72]:
x = DataFrameBuild(ctx.spark)

DF1

In [73]:
x = DataFrameBuild(ctx.spark)
num = 500
df1 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                            ['index','passwords','quarter','month','price'])

DF2

In [74]:
x = DataFrameBuild(ctx.spark)
num = 500
df2 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                             ['index','passwords','quarter','month','price'])

In [75]:
df1.limit(10).toPandas()

,index,passwords,quarter,month,price
0,1,ohnzfgbi,3,12,"5,926.64"
1,2,bxosgnmy,1,4,"4,124.54"
2,3,syaqohqi,3,11,"5,729.71"
3,4,kltwiihc,1,6,906.52
4,5,qqhvhvyi,1,8,866.57
5,6,jmivslqu,3,6,"6,875.55"
6,7,nnlcoatb,1,9,"7,585.08"
7,8,xmmcqobm,3,6,"2,244.51"
8,9,dyrhslbd,1,8,"8,728.90"
9,10,tqkbzfbd,1,7,"5,038.72"


In [76]:
df2.limit(10).show()

+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+
|    1| vcukpbff|      4|    6| 5541.234393163349|
|    2| efhqbbzq|      3|    7| 2293.619015284052|
|    3| bztlqpgs|      3|    7| 6826.345542082305|
|    4| kvoqzvbc|      2|    8| 9793.813883245322|
|    5| ycyqpfsg|      3|    6|325.92287334774704|
|    6| ogqbdavo|      3|    2| 6328.430623807999|
|    7| bbrfynno|      1|    3|  8494.68270334824|
|    8| mlssfyup|      2|    5| 4987.541574817552|
|    9| ivaipmzm|      1|   11| 4200.649806516106|
|   10| ndgeytdj|      4|    7|1182.8858401941977|
+-----+---------+-------+-----+------------------+



## Join df1 and df2 on indices.

In [13]:
df_index = df1.join(df2,df1.index == df2.index)
df_index = df1.join(df2,df1["index"] == df2["index"])

In [14]:
%%time
df_index.cache()

Wall time: 257 ms


DataFrame[index: bigint, passwords: string, quarter: bigint, month: bigint, price: double, index: bigint, passwords: string, quarter: bigint, month: bigint, price: double]

In [15]:
df_index.show(20)

+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|   26| yybawdph|      3|    3| 4101.289824424533|   26| kirgbycw|      1|    6| 8178.847233321894|
|   29| zstmsafe|      1|   10|3164.9332402134755|   29| soliyidw|      3|    4|  9937.78435562847|
|  474| yobwpbgf|      3|    8| 8737.929693899026|  474| yljidurj|      1|    4|2931.9652989530264|
|   65| ebcrgbeo|      3|    3|  6312.95896365517|   65| lglmlscw|      2|   12|3652.9064903034446|
|  191| rbphfoio|      1|    7| 8246.789676823599|  191| hhdeutpl|      1|    1| 4525.191395447187|
|  418| iebykzcr|      1|    2| 4247.362931335745|  418| hswienxm|      4|    8| 7913.534031722076|
|  222| kboilakn|      2|    7|2045.2625076775732|  222| ecucxnjt|      1|   11|  7360.19458119175|


In [16]:
df_index.columns

['index',
 'passwords',
 'quarter',
 'month',
 'price',
 'index',
 'passwords',
 'quarter',
 'month',
 'price']

## Join df1 and df2 on indices. (round 2)

In [ ]:
df_i2 = df1.alias("o")\
.join(df2.alias("t"),col("o.index") == col("t.index"))

# df_index = df1.join(df2,df1["index"] == df2["index"])

In [ ]:
%%time
df_i2.cache()

In [ ]:
df_i2.show(20)

In [ ]:
df_i2.columns

## Create a new column calculated from current column values.

In [14]:
df3 = df1\
.filter(col("month") == 9)\
.filter(col("month") == 8)

In [22]:
df4 = df1\
.filter((col("month") == 9) | 
        (col("month") == 8) |
        (col("quarter") == 3)
       )

In [15]:
df3.limit(8).toPandas()

,index,passwords,quarter,month,price


In [24]:
df4.limit(32).toPandas()

,index,passwords,quarter,month,price
0,1,clcnnvyv,2,8,"6,022.08"
1,4,dbpxfrqw,3,12,"8,172.45"
2,5,bdqlofhq,2,9,"3,720.03"
3,11,jkztjrrm,1,8,"6,926.03"
4,12,jdailwpz,3,9,"2,013.15"
5,15,lqgkktxc,3,1,"2,504.63"
6,18,toqcmpak,3,11,"8,211.26"
7,21,nzwehhzg,3,4,"2,402.05"
8,22,awqrcast,3,3,538.21
9,26,msfudwue,3,12,"2,483.86"


In [23]:
df4.count()

180

In [40]:
df5 = df4\
.groupBy("month", "quarter")\
.agg(avg("price"))\
.orderBy("month")

In [42]:
df5.count()

18

In [43]:
df6 = df5.filter(col("quarter") == 3)

In [46]:
df6.limit(10).toPandas()

,month,quarter,avg(price)
0,1,3,"6,097.54"
1,2,3,"7,020.26"
2,3,3,"5,117.60"
3,4,3,"4,486.07"
4,5,3,"3,399.59"
5,6,3,"5,939.97"
6,7,3,"6,539.49"
7,8,3,"5,642.59"
8,9,3,"3,891.82"
9,10,3,"4,920.09"


In [41]:
df5.limit(50).toPandas()

,month,quarter,avg(price)
0,1,3,"6,097.54"
1,2,3,"7,020.26"
2,3,3,"5,117.60"
3,4,3,"4,486.07"
4,5,3,"3,399.59"
5,6,3,"5,939.97"
6,7,3,"6,539.49"
7,8,3,"5,642.59"
8,8,2,"4,350.95"
9,8,4,"6,849.49"


In [37]:
# df4.groupby("quarter")\
# .agg(count("quarter")).show()

df4\
.withColumn("eps_avg",when((col("month") == 9),sum(df4.filter(col("month") == 9)['price'])).otherwise(0))

            
# .filter(col("quarter") == 3 & col("month") != 8
#     .agg(when(, count("quarter")))


AnalysisException: "grouping expressions sequence is empty, and '`index`' is not an aggregate function. Wrap '(CASE WHEN (`month` = CAST(9 AS BIGINT)) THEN sum(`price`) ELSE CAST(0 AS DOUBLE) END AS `eps_avg`)' in windowing function(s) or wrap '`index`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [index#0L, passwords#1, quarter#2L, month#3L, price#4, CASE WHEN (month#3L = cast(9 as bigint)) THEN sum(price#4) ELSE cast(0 as double) END AS eps_avg#119]\n+- Filter (((month#3L = cast(9 as bigint)) || (month#3L = cast(8 as bigint))) || (quarter#2L = cast(3 as bigint)))\n   +- LogicalRDD [index#0L, passwords#1, quarter#2L, month#3L, price#4], false\n"

# df1 cont'd

In [85]:
df1.count()

500

In [65]:
df1.limit(10).toPandas()

,index,passwords,quarter,month,price
0,1,kzkbsjxl,2,11,"5,644.58"
1,2,srntqhqq,3,5,"9,814.11"
2,3,lmfxfoci,2,3,"1,089.28"
3,4,eydoutop,1,10,"6,535.37"
4,5,qlhgpuut,3,9,"5,357.19"
5,6,prvlsexg,3,11,9.55
6,7,zbfxzkzn,3,10,"1,303.85"
7,8,jsdpvchc,3,10,"6,359.49"
8,9,ophmvozn,3,5,"2,002.10"
9,10,lwzxioze,1,10,"2,975.93"


In [69]:
w_priceperquarter = Window\
.orderBy(col("month").desc())\
.rangeBetween(4, 6)

In [ ]:
windowSpec = \
  Window 
    .partitionBy(df['category']) \
    .orderBy(df['revenue'].desc()) \
    .rangeBetween(-sys.maxsize, sys.maxsize)

In [72]:
df1.withColumn("avg-price",w_priceperquarter(avg('price'))).show()

TypeError: 'WindowSpec' object is not callable

In [ ]:
w = (Window.orderBy(F.col("timestampGMT").cast('long')).rangeBetween(-days(7), 0))

df = df.withColumn('rolling_average', F.avg("dollars").over(w))

In [87]:
w1 = Window\
.orderBy(col("index"))\
.rangeBetween(

TypeError: rangeBetween() missing 1 required positional argument: 'end'

# Window testing

In [80]:
winQM = Window\
.partitionBy('quarter')\
.partitionBy('month')\
.orderBy("quarter","month")

winQM2 = Window\
.partitionBy('month')\
.partitionBy('quarter')\
.orderBy("quarter","month")

winQM3 = Window\
.partitionBy('quarter', 'month')\
.orderBy("quarter","month")


winQ = Window\
.partitionBy('quarter')\
.orderBy("quarter","month")

winM = Window\
.partitionBy("month")\
.orderBy("quarter","month")


byMonth = Window\
.orderBy("month")\
.partitionBy('month')\
.partitionBy('quarter')

In [67]:
df_Q = df1.withColumn("avg_price", mean("price").over(winQ))
# df_Q.show(12)
df_Q.sample(0.1).show(20)

+-----+---------+-------+-----+------------------+------------------+
|index|passwords|quarter|month|             price|         avg_price|
+-----+---------+-------+-----+------------------+------------------+
|  441| xhruzkww|      1|    1| 5198.690268804251|4990.4177824743365|
|  401| iccfqpiv|      1|    3| 8895.113007113754| 5029.899801333232|
|  437| riphelwq|      1|    3|3516.8071596013638| 5029.899801333232|
|  146| zuhlmlfr|      1|    4|1133.8677280772959| 4868.552179284961|
|  191| rbphfoio|      1|    7| 8246.789676823599| 4958.030360022241|
|  297| akmwrrml|      1|    9| 1432.952340223278| 5003.645762335367|
|  257| ggzntort|      3|    1| 7402.294186189028|6468.0302799844885|
|   26| yybawdph|      3|    3| 4101.289824424533| 5340.272721129356|
|  376| odtthcgs|      3|    3| 4894.791671891981| 5340.272721129356|
|   86| qnyrnzxf|      3|    4| 2751.261862138721| 5473.155379283507|
|   88| sxwfcwgp|      3|    4| 9386.868898997645| 5473.155379283507|
|  298| nlxeaovh|   

In [68]:
df_M = df1.withColumn("avg_price", mean("price").over(winM))
# df_M.show(12)
df_M.sample(0.1).show(20)

+-----+---------+-------+-----+------------------+------------------+
|index|passwords|quarter|month|             price|         avg_price|
+-----+---------+-------+-----+------------------+------------------+
|  191| rbphfoio|      1|    7| 8246.789676823599|  4426.71917345324|
|  222| kboilakn|      2|    7|2045.2625076775732| 4767.665634438058|
|  185| mkckkrxk|      1|    6| 6779.021448831084| 5479.341144568709|
|  219| pppbijgt|      1|    6| 7903.928260724644| 5479.341144568709|
|   14| dfqykcbg|      4|    9| 8368.375631710302| 4781.640877243091|
|  197| lujgiqiy|      1|    5|1600.9123494530443| 5131.325026252701|
|  271| yueeooem|      1|    5| 818.5740661364449| 5131.325026252701|
|   35| srwsreit|      2|    5|2390.2220905364534| 4932.332128113493|
|    3| htfgzhia|      3|    5|6726.2263238776795| 4722.662702885805|
|   79| rxltrvsh|      4|    5| 6130.945585246142| 4620.811458783582|
|  435| hzivoqiz|      4|    5|  4864.37886327801| 4620.811458783582|
|  147| bezkrwey|   

In [69]:
df_QM1 = df_Q.withColumn("avg_priceM", mean("price").over(winM))
# df_QM1.show(12)
df_QM1.sample(0.1).show(20)

+-----+---------+-------+-----+------------------+------------------+------------------+
|index|passwords|quarter|month|             price|         avg_price|        avg_priceM|
+-----+---------+-------+-----+------------------+------------------+------------------+
|  248| vmqgpcxf|      3|    7| 5053.231039789373| 5262.825022487481| 5221.352705421936|
|  287| vmampwua|      3|    7| 2522.781929344662| 5262.825022487481| 5221.352705421936|
|  318| haeismqz|      4|    7| 6049.653019381443| 4585.701664295067|  4854.81323099371|
|  321| vtnyonfe|      2|    6|3080.4980163234472| 4853.700445399636| 4928.788891625552|
|  355| ylruchxk|      2|    6|1304.7336916720033| 4853.700445399636| 4928.788891625552|
|  183| bszmwvum|      3|    6| 7329.526921985802| 5113.754943505666| 4835.678037732626|
|  168| zcmbxxoi|      4|    6| 8044.718817245886| 4674.681710188344| 4893.063983706119|
|  338| fmkkdfos|      4|    6|1821.1113581416828| 4674.681710188344| 4893.063983706119|
|   61| rxpronch|    

In [70]:
# df_a1 = df1.withColumn("avg_price", mean("price").over(winQM)).sample(0.1).orderBy("quarter","month").show(12)

# Note to self:
- Why do I prefer Window over GroupBy?
- Ans: i'd have to join later
- Ans: if I want to filter by 2 later columns.. but currently this would have the negative values in there
- Ans: alternatively, if I filter by age, then I can't get many ages .. or go back and filter on multiple columns later

- example: month 1, quarter 2, if password starts with h
- filter(startswith(h)) & month = 1 & quarter = 2)

In [60]:
df_gb1 = df1.groupBy("quarter","month").agg(mean("price")).sample(0.1).show(32)

+-------+-----+-----------------+
|quarter|month|       avg(price)|
+-------+-----+-----------------+
|      2|    5|4763.953521995699|
|      3|    8|7825.547969734452|
|      1|    4|4472.517106984661|
|      2|    3|5295.922507614247|
|      2|   12|5115.058605467251|
|      1|   10|4517.402583645931|
|      4|    1|5386.766857815846|
|      2|    4|5467.194414802793|
|      3|    9|4321.776549365829|
|      4|   11| 5306.08865537361|
|      1|   12|4691.809543727584|
+-------+-----+-----------------+



## month then quarter

In [86]:
df_QM_a1 = df1.withColumn("avg_priceM", mean("price").over(winM))
df_QM_a2 = df_QM_a1.withColumn("avg_priceQ", mean("price").over(winQ))
df_QM_a3 = df_QM_a2.withColumn("avg_priceQM", mean("price").over(winQM))
df_QM_a4 = df_QM_a3.withColumn("avg_priceQM2", mean("price").over(winQM2))
df_QM_a5 = df_QM_a4.withColumn("avg_priceQM3", mean("price").over(winQM3))

# df_QM_a4.show(12)
df_QM_a5.sample(0.1).limit(32).toPandas()

,index,passwords,quarter,month,price,avg_priceM,avg_priceQ,avg_priceQM,avg_priceQM2,avg_priceQM3
0,467,ojoytpkb,1,2,"9,830.60","3,887.23","4,079.88","3,887.23","4,079.88","3,887.23"
1,50,yflqqspv,1,4,"5,886.54","4,869.66","4,631.30","4,869.66","4,631.30","4,869.66"
2,174,cjwztyab,1,8,"8,190.26","6,267.96","5,224.18","6,267.96","5,224.18","6,267.96"
3,53,kmvgpnnh,1,9,"2,286.63","5,814.37","5,312.71","5,814.37","5,312.71","5,814.37"
4,395,ewwiqgdl,1,9,"8,720.11","5,814.37","5,312.71","5,814.37","5,312.71","5,814.37"
5,316,bfoarlog,1,10,"3,957.11","4,028.62","5,170.04","4,028.62","5,170.04","4,028.62"
6,379,kusabwsp,1,12,93.99,"4,569.25","5,210.80","4,569.25","5,210.80","4,569.25"
7,419,bsjsekbl,3,1,"7,599.35","4,553.52","3,619.04","4,553.52","3,619.04","3,619.04"
8,181,uyfcwqnw,3,3,"2,609.72","5,183.65","4,942.89","5,183.65","4,942.89","5,749.47"
9,447,gljeuepl,3,4,"9,984.23","5,150.34","4,957.91","5,150.34","4,957.91","5,078.11"


## quarter then month

In [94]:
dq1 = df1\
.withColumn("Avg_priceQ", mean("price").over(winQ))\
.withColumn("Avg_priceM", mean("price").over(winM))\
.withColumn("Avg_priceQM", mean("price").over(winQM))\
.withColumn("Avg_priceQM3", mean("price").over(winQM3))

dm1 = df1\
.withColumn("Avg_priceM", mean("price").over(winM))\
.withColumn("Avg_priceQ", mean("price").over(winQ))\
.withColumn("Avg_priceQM", mean("price").over(winQM))\
.withColumn("Avg_priceQM3", mean("price").over(winQM3))


# df_QM_a4 = df_QM_a3.withColumn("avg_priceQM2", mean("price").over(winQM2))
# df_QM_a5 = df_QM_a4.withColumn("avg_priceQM3", mean("price").over(winQM3))

# # df_QM_a4.show(12)
# df_QM_a5.sample(0.1).limit(32).toPandas()

In [99]:
dq1.sample(0.1).limit(100).toPandas()

,index,passwords,quarter,month,price,Avg_priceQ,Avg_priceM,Avg_priceQM,Avg_priceQM3
0,67,jzdoedwv,2,7,"9,316.75","4,968.15","5,153.14","5,153.14","4,484.17"
1,80,ahtbhybc,3,7,"8,882.99","4,733.80","5,025.37","5,025.37","4,748.55"
2,496,empfguux,3,7,"3,629.10","4,733.80","5,025.37","5,025.37","4,748.55"
3,64,cibujdor,4,7,"4,384.33","5,416.12","5,227.33","5,227.33","5,775.49"
4,277,azcelwdq,4,7,"4,013.80","5,416.12","5,227.33","5,227.33","5,775.49"
5,463,inspkjrb,4,7,"5,096.08","5,416.12","5,227.33","5,227.33","5,775.49"
6,149,zrmyfbcv,1,6,"7,827.66","4,843.45","5,825.45","5,825.45","5,825.45"
7,140,hebegeyq,2,6,"6,769.04","5,038.29","5,109.60","5,109.60","4,662.20"
8,118,qrrcyiop,4,6,"8,700.59","5,332.27","4,975.84","4,975.84","6,567.06"
9,292,mihbhlbp,4,6,"3,772.66","5,332.27","4,975.84","4,975.84","6,567.06"


In [100]:
dm1.sample(0.1).limit(100).toPandas()

,index,passwords,quarter,month,price,Avg_priceM,Avg_priceQ,Avg_priceQM,Avg_priceQM3
0,36,tcwjbcbc,1,7,"5,277.66","5,571.24","5,044.22","5,571.24","5,571.24"
1,191,qnnfaarz,1,7,"5,462.44","5,571.24","5,044.22","5,571.24","5,571.24"
2,59,almbiecf,4,7,"8,230.19","5,227.33","5,416.12","5,227.33","5,775.49"
3,64,cibujdor,4,7,"4,384.33","5,227.33","5,416.12","5,227.33","5,775.49"
4,182,gwtipjin,4,7,"9,881.87","5,227.33","5,416.12","5,227.33","5,775.49"
5,219,ojamycws,2,6,"3,729.15","5,109.60","5,038.29","5,109.60","4,662.20"
6,317,yebbxgkp,2,6,"7,428.71","5,109.60","5,038.29","5,109.60","4,662.20"
7,440,xdiuvhbo,2,6,779.63,"5,109.60","5,038.29","5,109.60","4,662.20"
8,444,nebdncgo,2,6,"3,753.97","5,109.60","5,038.29","5,109.60","4,662.20"
9,233,pmjhyzgz,3,6,"4,046.95","4,302.63","4,731.07","4,302.63","3,495.65"


In [101]:
dg1 = df1\
.groupBy("quarter","month")\
.agg(avg("price"))\
.orderBy("quarter","month")

In [98]:
dg1.limit(100).toPandas()

,quarter,month,avg(price)
0,1,1,"4,272.54"
1,1,2,"3,887.23"
2,1,3,"5,275.23"
3,1,4,"4,869.66"
4,1,5,"7,570.77"
5,1,6,"5,825.45"
6,1,7,"5,571.24"
7,1,8,"6,267.96"
8,1,9,"5,814.37"
9,1,10,"4,028.62"


## Example 1.

## Example 2.

### month then quarter

In [ ]:
+-----+---------+-------+-----+------------------+------------------+------------------+------------------+------------------+
|index|passwords|quarter|month|             price|        avg_priceM|        avg_priceQ|       avg_priceQM|      avg_priceQM2|
+-----+---------+-------+-----+------------------+------------------+------------------+------------------+------------------+
|  145| jhfxegcs|      1|    4|3293.3090538514407| 4869.663346376393| 4631.304900548585| 4869.663346376393| 4631.304900548585|
|  456| bnunfcex|      1|    6| 8036.659841674585| 5825.451313405875| 4843.452551806632| 5825.451313405875| 4843.452551806632|
|  188| fgwndujf|      1|    7|  9285.08697130339| 5571.243320190696|5044.2224189470635| 5571.243320190696|5044.2224189470635|
|  257| iolobnir|      1|    7| 5844.348712681021| 5571.243320190696|5044.2224189470635| 5571.243320190696|5044.2224189470635|
|  174| cjwztyab|      1|    8| 8190.256431180303| 6267.964915063347| 5224.184550728871| 6267.964915063347| 5224.184550728871|
|  359| ullqzdgx|      1|    8| 7187.726778386144| 6267.964915063347| 5224.184550728871| 6267.964915063347| 5224.184550728871|
|  457| zmsethvs|      1|   10|3513.1166209239095|4028.6208085540793| 5170.035492161211|4028.6208085540793| 5170.035492161211|

### quarter then month

In [ ]:
+-----+---------+-------+-----+------------------+------------------+------------------+------------------+------------------+
|index|passwords|quarter|month|             price|        avg_priceQ|        avg_priceM|       avg_priceQM|      avg_priceQM2|
+-----+---------+-------+-----+------------------+------------------+------------------+------------------+------------------+
|   34| higrpjmt|      1|    4| 6339.108660097459| 4631.304900548585| 4869.663346376393| 4869.663346376393| 4631.304900548585|
|  109| wyxinfwk|      1|    4| 2374.710542665626| 4631.304900548585| 4869.663346376393| 4869.663346376393| 4631.304900548585|
|  145| jhfxegcs|      1|    4|3293.3090538514407| 4631.304900548585| 4869.663346376393| 4869.663346376393| 4631.304900548585|
|   31| qzczigak|      1|    6| 4058.168883900758| 4843.452551806632| 5825.451313405875| 5825.451313405875| 4843.452551806632|
|   10| tqkbzfbd|      1|    7| 5038.718599778232|5044.2224189470635| 5571.243320190696| 5571.243320190696|5044.2224189470635|
|  361| ndjryrjb|      1|    7|5801.8506640415435|5044.2224189470635| 5571.243320190696| 5571.243320190696|5044.2224189470635|
|  487| iwmeldtm|      1|    8| 6943.407184649127| 5224.184550728871| 6267.964915063347| 6267.964915063347| 5224.184550728871|
|   70| hkjhieed|      1|   10|  2859.92825187243| 5170.035492161211|4028.6208085540793|4028.6208085540793| 5170.035492161211|

# byMonth

In [110]:
df_w1 = df1.withColumn('new_win', avg("price").over(byMonth))

In [111]:
df_w1.limit(50).toPandas()

,index,passwords,quarter,month,price,new_win
0,35,vuirsthb,1,1,"3,933.36","2,811.85"
1,99,umtmvgmb,1,1,"1,887.00","2,811.85"
2,131,ktstnwst,1,1,701.86,"2,811.85"
3,237,ftncbemr,1,1,"8,576.77","2,811.85"
4,255,dwkpafwn,1,1,59.76,"2,811.85"
5,281,pkxlxxjb,1,1,630.41,"2,811.85"
6,287,tdyqzcle,1,1,"1,831.79","2,811.85"
7,310,oyerfrch,1,1,"6,388.64","2,811.85"
8,346,rsmjocpc,1,1,"2,298.90","2,811.85"
9,363,orgtushl,1,1,"1,828.20","2,811.85"


In [108]:
df_w1.count()

500

In [114]:
df_w1.select("new_win").distinct().limit(100).toPandas()

,new_win
0,"5,253.71"
1,"4,739.01"
2,"5,283.91"
3,"5,280.76"
4,"5,153.14"
5,"4,660.07"
6,"5,141.04"
7,"4,818.53"
8,"5,107.83"
9,"5,658.42"
